In [9]:
pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.9 MB 5.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
import robustbench
from robustbench.utils import load_model
from tqdm import tqdm
import os
import torch
from autoattack import AutoAttack
from robustbench.data import _load_dataset
from torchvision import transforms, datasets
import robustbench
from robustbench.utils import load_model
import os
import torch
import multiprocessing as mp
import pandas as pd
import sys
sys.path.append("/gavrikov/pytorch-pretrained-cnns")
import data
import models
from glob import glob
from collections import defaultdict
import onnxruntime
import numpy as np
import onnx
import struct
from joblib import Parallel, delayed

In [11]:
dataset_to_model_dict = defaultdict(list)

for path in glob("/ssd1/robustness_onnx_zoo/*/*.onnx"):
    ds = "imagenet"
    if "_cifar10_" in path:
        ds = "cifar10"
    elif "_cifar100_" in path:
        ds = "cifar100"
    
    dataset_to_model_dict[ds].append(path)

In [39]:
evaluation = []

def inference_accuracy(model_path, x, y):
    try:   
        ort_session = onnxruntime.InferenceSession(model_path)
        ort_inputs = {ort_session.get_inputs()[0].name: x}
        acc = (ort_session.run(None, ort_inputs)[0].argmax(axis=1) == y).mean()
        return (model_path, dataset, acc)
    except Exception as e:
        return (model_path, dataset, str(type(e)) + str(e))

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

for dataset, model_paths in dataset_to_model_dict.items():
    
    if dataset == "cifar10":
        ds = data.CIFAR10Data("/data/datasets/cifar10", 256, 8, normalize=False)
    elif dataset == "cifar100":
        ds = data.CIFAR100Data("/data/datasets/cifar100", 256, 8, normalize=False)
    elif dataset == "imagenet":
        ds = data.ImageNet1kData("/data/datasets/imagenet1k", 256, 8, normalize=False)
    
    x, y = next(iter(ds.test_dataloader()))
    x = x.numpy()
    y = y.numpy()
    
    results = Parallel(n_jobs=16)(delayed(inference_accuracy)(model_path, x, y) for model_path in model_paths)
    evaluation.extend(results)

Files already downloaded and verified
Files already downloaded and verified


In [41]:
pd.DataFrame(evaluation, columns=["ID", "Dataset", "Single Batch Eval"]).set_index("ID").to_csv("val_performance.csv")